In [1]:
import numpy as np
import pandas as pd
import glob, os, time

In [3]:
data_root = '/oak/stanford/groups/mrivas/private_data/ukbb/24983/'
out_dir = '/oak/stanford/groups/mrivas/dev-ukbb-tools/phenotypes/9796/20003/'
src = glob.glob(os.path.join(data_root, 'phenotypedata/download/9796/newest/ukb*.tab'))[0]
input_data = pd.read_table(src, 
                           usecols=lambda x:'20003' in x.split('.') or x == 'f.eid', 
                           index_col='f.eid', 
                           dtype='str')
input_data.head()

,f.20003.0.0,f.20003.0.1,f.20003.0.2,f.20003.0.3,f.20003.0.4,f.20003.0.5,f.20003.0.6,f.20003.0.7,f.20003.0.8,f.20003.0.9,...,f.20003.2.38,f.20003.2.39,f.20003.2.40,f.20003.2.41,f.20003.2.42,f.20003.2.43,f.20003.2.44,f.20003.2.45,f.20003.2.46,f.20003.2.47
f.eid,,,,,,,,,,,,,,,,,,,,,
4646215,2038460150,1141157402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5007659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1038854,1141146188,1140875840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5441319,1140875408,1140868226,1140879760,1141176662,1187,1141168318,1140923346,1189,1140923348,1140884488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5743544,1140866738,1141194794,1140871236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
phe_ref = dict([line.split() for line in open('MED_input.tsv')])
phe_ref.items()[:10]

[('1140871092', 'salsalate'),
 ('1140874590', 'vermox_100mg/5ml_suspension'),
 ('1140874596', 'piperazine'),
 ('1140860882', 'cilazapril'),
 ('1140864502', 'testotop_tts_15mg_transdermal_patch'),
 ('1140879556', 'mianserin'),
 ('1140879554', 'estramustine'),
 ('1140879552', 'maprotiline'),
 ('1140875350', 'mobiflex_20mg_tablet'),
 ('1140875420', 'baclofen')]

In [5]:
popref = {pop:pd.read_table(os.path.join(data_root, 'sqc/population_stratification/ukb24983_{}.phe'.format(pop)),
                            index_col=0).iloc[:,0].to_dict() 
              for pop in ('white_british', 'african', 'e_asian', 's_asian')}

In [6]:
# useful for phenotype info
header="#GBE_ID GBE_NAME FIELD TABLE BASKET APP_ID N N_GBE N_AFR N_EAS N_SAS SOURCE DATE PATH".replace(" ", "\t")

# iterate through reference info -- the notebook is truncated for brevity
#                                -- the script will need 2 days to run everything
for i,(code,name) in enumerate(phe_ref.items()[:5]):
    t=time.time()
    # load up data -- show some examples in the notebook
    new_phe = input_data.apply(lambda x:code in x.tolist(), axis=1)
    print(new_phe.describe())
    new_phe = new_phe.to_dict()
    out_phe = os.path.join(out_dir, 'MED'+code+'.phe')
    # write out to file above
    with open(out_phe, 'w') as o:
        o.write("\n".join(map(lambda x:' '.join((str(x[0]),str(x[0]),'2' if x[1] else '1')), new_phe.items())))
    # and make an info file
    with open(out_phe+'.info', 'w') as o:
        o.write(header+'\n')
        o.write('\t'.join(['MED'+code, 
                           name, 
                           '20003', 
                           os.path.splitext(os.path.basename(src))[0].replace('ukb',''), 
                           '9796', 
                           '24983', 
                           str(sum(new_phe.values()))] + 
                          [str(len(set(popref[p].keys()) & set([k for k,v in new_phe.items() if v]))) for p in 
                           ['white_british', 'african', 'e_asian', 's_asian']] + 
                          ['MED_definitions.py', 
                           str(pd.Timestamp.today()).split()[0], 'MED'+code+'.phe']
                         ) + '\n')
    print(i,time.time()-t)
        

count     502543
unique         1
top        False
freq      502543
dtype: object


IOError: [Errno 2] No such file or directory: '/oak/stanford/groups/mrivas/dev-ukbb-tools/phenotyping/9796/20003/MED1140871092.phe'